# This is 3.1

In [3]:
import tensorflow as tf
from tensorflow import keras
import argparse
import os
import numpy as np
from board import D4
import adafruit_dht
import pandas as pd
import datetime
import time
import argparse

In [4]:



parser = argparse.ArgumentParser()
parser.add_argument('--model', type=str, required=True)
args = parser.parse_args(['--model=ex1'])

#converter = tf.lite.TFLiteConverter.from_saved_model('home/pi/WORK_DIR/notebooks/Ex_Frank/LABS/Lab3/saved_models/')
#tflite_model = converter.convert()
# The value returned by convert is a “bytes”
# object which contains the serialized model, and
# can be directly written to a binary file



In [5]:
interpreter = tf.lite.Interpreter(model_path="{}.tflite".format(args.model))
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
                                  
print("Number of inputs:", len(input_details))
print("Number of outputs:", len(output_details))
print("Input name:", input_details[0]['name'])
print("Input shape:", input_details[0]['shape'])

Number of inputs: 1
Number of outputs: 1
Input name: serving_default_x:0
Input shape: [1 6 2]


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [8]:
window = np.zeros([1, 6, 2], dtype=np.float32)
true_values = np.zeros(2, dtype=np.float32)


dht_device = adafruit_dht.DHT11(D4)

MEAN = np.array([9.107597, 75.904076], dtype=np.float32)
STD = np.array([ 8.654227, 16.557089], dtype=np.float32)

while True:
    for i in range(7):
        temperature = dht_device.temperature
        humidity = dht_device.humidity
        
        if i<6:
            window[0, i, 0] = np.float32(temperature)
            window[0, i, 1] = np.float32(humidity)
        if i==6:
            true_values[0] = np.float32(temperature)
            true_values[1] = np.float32(humidity)
            
            window = (window - MEAN) / STD
            interpreter.set_tensor(input_details[0]['index'], window)
            interpreter.invoke()
            predicted = interpreter.get_tensor(output_details[0]['index'])
            print(predicted)
            
            print('Measured values: {:.1f}.{:.1f}'.format(true_values[0], true_values[1]))
            print('Predicted: {:.1f},{:.1f}'.format(predicted[0, 0], predicted[0, 1]))        
        time.sleep(0.2)


[[37.2885]]
Measured values: 36.0.15.0


IndexError: index 1 is out of bounds for axis 1 with size 1

In [26]:
class WindowGenerator:
    def __init__(self, input_width, label_options, mean, std):
        self.input_width = input_width  # the number of samples contained in a single window
        self.label_options = label_options
        self.mean = tf.reshape(tf.convert_to_tensor(mean), [1, 1, 2])  # sec arg is the shape and during training the first dim is batch
        self.std = tf.reshape(tf.convert_to_tensor(std), [1, 1, 2])

    def split_window(self, features):
        # here the assumption is that features already contains 7 values
        inputs = features[:, :-1, :]  # we leave the last one as label, remember we have the batch at the first dim

        if self.label_options < 2:
            labels = features[:, -1, self.label_options]
            labels = tf.expand_dims(labels, -1)
            num_labels = 1
        else:
            labels = features[:, -1, :]
            num_labels = 2

        inputs.set_shape([None, self.input_width, 2])  # we set the batch dim as None because it will be needed later
        labels.set_shape([None, num_labels])  # same for labels

        return inputs, labels

    def normalize(self, features):
        features = (features - self.mean) / (self.std + 1.e-6)  # we add a small quantity to avoid divisions by zero

        return features

    def preprocess(self, features):
        inputs, labels = self.split_window(features)
        inputs = self.normalize(inputs)  # we don't need to normalize labels

        return inputs, labels

    def make_dataset(self, data, train):
        ds = tf.keras.preprocessing.timeseries_dataset_from_array( # this is what we need
                data=data,
                targets=None,
                sequence_length=self.input_width+1,  # + label
                sequence_stride=1,
                batch_size=32)  # we'll try to change batch size later
        ds = ds.map(self.preprocess)  # the preprocess will be applied to every batch
        ds = ds.cache()

        if train is True:
            ds = ds.shuffle(100, reshuffle_each_iteration=True) # 100 is the shuffling buffer size

        return ds


class customMAE(keras.metrics.Metric):
    def __init__(self, name='custom_MAE', **kwargs):
        super().__init__(name=name, **kwargs)
        # MAE: we need the sum --> total, and the count --> count to compute the mean
        self.total = self.add_weight('total', initializer='zero', shape=(2,))
        self.count = self.add_weight('count', initializer='zeros')

    # We have to iterate over all the dataset and update the state vars
    # This is computed at every batch
    def update_state(self, y_true, y_pred, sample_weight=None):
        error = tf.abs(y_pred-y_true)
        error = tf.reduce_mean(error, axis=0)
        self.total.assign_add(error)
        self.count.assign_add(1.)
        return

    def reset_states(self):
        self.count.assign(tf.zeros_like(self.count))
        self.total.assign(tf.zeros_like(self.total))
        return

    # after we have updated for all the dataset we return the result
    def result(self):
        result = tf.math.divide_no_nan(self.total, self.count)
        return result


In [28]:
# let's adapt our input
generator = WindowGenerator(6, 2, input_data.mean(axis=0), input_data.std(axis=0))
input_ds = generator.make_dataset(input_data, True)

TypeError: in user code:

    <ipython-input-26-ae34ae061efc>:32 preprocess  *
        inputs = self.normalize(inputs)  # we don't need to normalize labels
    <ipython-input-26-ae34ae061efc>:26 normalize  *
        features = (features - self.mean) / (self.std + 1.e-6)  # we add a small quantity to avoid divisions by zero
    /home/pi/WORK_DIR/py37/lib/python3.7/site-packages/tensorflow/python/ops/math_ops.py:1141 binary_op_wrapper
        raise e
    /home/pi/WORK_DIR/py37/lib/python3.7/site-packages/tensorflow/python/ops/math_ops.py:1125 binary_op_wrapper
        return func(x, y, name=name)
    /home/pi/WORK_DIR/py37/lib/python3.7/site-packages/tensorflow/python/util/dispatch.py:201 wrapper
        return target(*args, **kwargs)
    /home/pi/WORK_DIR/py37/lib/python3.7/site-packages/tensorflow/python/ops/math_ops.py:527 subtract
        return gen_math_ops.sub(x, y, name)
    /home/pi/WORK_DIR/py37/lib/python3.7/site-packages/tensorflow/python/ops/gen_math_ops.py:10466 sub
        

In [45]:
input_data = np.expand_dims(input_data, axis=0)
input_data = tf.convert_to_tensor(input_data, dtype=tf.float32)

NameError: name 'float32' is not defined

In [50]:
input_data.shape

TensorShape([1, 6, 2])

In [49]:
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()  # here the inference takes place (NN is executed)

my_output = interpreter.get_tensor(output_details[0]['index'])  # returns value contained in the out buffer as np arrays
print("Output:", my_output)  

Output: [[471.6208]]


In [58]:
test_data

Temperature    41
Humidity       10
Name: 6, dtype: object

# Ex 3.2

In [44]:
ls home/pi/WORK_DIR

Ex_Ciccio/    nohup.out   rpi_requirements.txt
Ex_Frank/     notebooks/  tensorflow-2.3.0-cp37-none-linux_armv7l.whl
Ex_Gianluca/  py37/


In [89]:
import argparse
import numpy as np
import pyaudio
import tensorflow as tf
import time
import wave
from io import BytesIO
from scipy.io import wavfile
from scipy import signal

model = 'model'
    
interpreter = tf.lite.Interpreter(model_path='./models/{}.tflite'.format(model))
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Number of inputs:", len(input_details))
print("Number of outputs:", len(output_details))
print("Input name:", input_details[0]['name'])
print("Input shape:", input_details[0]['shape'])
print("Output shape:", output_details[0]['shape'])



['.ipynb_checkpoints' 'up' 'stop' 'no' 'go' 'left' 'down' 'yes' 'right']
Audio shape without padding (None,)
Audio shape with padding (16000,)


In [92]:
data_dir = "home/pi/WORK_DIR/notebooks/Ex_Gianlu/LABs/LAB 3/data/mini_speech_commands"

mfcc_bool = True

STFT_OPTIONS = {'frame_length': 256, 'frame_step': 128, 'mfcc': False}
MFCC_OPTIONS = {'frame_length': 640, 'frame_step': 320, 'mfcc': True,
                'lower_frequency': 20, 'upper_frequency': 4000, 'num_mel_bins': 40,
                'num_coefficients': 10}

if mfcc_bool:
    options = MFCC_OPTIONS
else:
    options = STFT_OPTIONS


filenames = tf.io.gfile.glob(str(data_dir) + '/*/*')  # Return list of files matching the pattern
filenames = tf.random.shuffle(filenames)
num_samples = len(filenames)

LABELS = np.array(tf.io.gfile.listdir(str(data_dir)))
LABELS = LABELS[LABELS != 'README.md'][:10]
print(LABELS)
 
    
generator = SignalGenerator(LABELS, 16000, **options)
input_dataset = generator.make_dataset(filenames[:10], True)

Number of inputs: 1
Number of outputs: 1
Input name: serving_default_conv2d_9_input:0
Input shape: [ 1 49 10  1]
Output shape: [1 8]


In [100]:
for e in input_dataset:
    elems = e[0]
    elems = tf.expand_dims(elems, axis=0)
    print(elems.shape)
    
    interpreter.set_tensor(input_details[0]['index'], elems)

    interpreter.invoke()  # here the inference takes place (NN is executed)

    my_output = interpreter.get_tensor(output_details[0]['index'])  # returns value contained in the out buffer as np arrays
    print("Output:", my_output)
    print("True labels:", e[1])
    
    break

(1, 49, 10, 1)
Output: [[-7.3472733 -2.3088493 -5.558369  -6.477327   3.7452102 -6.8825917
  -6.095357  -1.4488666]]
True labels: tf.Tensor(2, shape=(), dtype=int64)


In [97]:
for e in input_dataset:
    print(e[0].shape)

(49, 10, 1)
(49, 10, 1)
(49, 10, 1)
(49, 10, 1)
(49, 10, 1)
(49, 10, 1)
(49, 10, 1)
(49, 10, 1)
(49, 10, 1)
(49, 10, 1)
